## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

## Define the files to be loaded

In [2]:
# Participant number whose data should be loaded
participant_number = 2

# The version of the questionnaires to load
quest_version = 2

## Loading the pre-processed data

In [3]:
df = pd.read_csv('data/preprocessed/preprocessed_data_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df = df.drop(labels=['actual_day', 'actual_day.1'], axis=1)

df_oura = pd.read_csv('data/preprocessed/preprocessed_sleep_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df_oura = df_oura.drop(labels=['actual_day'], axis=1)

df_quest = pd.read_csv('data/preprocessed/preprocessed_questionnaires_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df_quest = df_quest.drop(labels=['actual_day'], axis=1)

In [4]:
# Remove these to try and fix the issues raised by LazyRegressor for them 
# example: 
# AdaBoostRegressor model failed to execute
# Found unknown categories ['[0.0, 51.0, 53.0, 53.0, 54.0, 53.0, ...
df = df.drop(labels=['hrv.items', 'heart_rate.items', 'sleep_phase_5_min', 'movement_30_sec'], axis=1)
df_oura = df_oura.drop(labels=['hrv.items', 'heart_rate.items', 'sleep_phase_5_min', 'movement_30_sec'], axis=1)

In [5]:
print(df.columns.tolist())

['0', 'average_breath', 'average_breath_variation', 'average_heart_rate', 'average_hrv', 'awake_time', 'bedtime_end_delta', 'bedtime_end_seconds', 'bedtime_start_delta', 'bedtime_start_seconds', 'contributors.deep_sleep', 'contributors.efficiency', 'contributors.latency', 'contributors.rem_sleep', 'contributors.restfulness', 'contributors.timing', 'contributors.total_sleep', 'deep_sleep_duration', 'efficiency', 'got_ups', 'latency', 'light_sleep_duration', 'lowest_heart_rate', 'lowest_heart_rate_time_offset', 'period', 'readiness.contributors.activity_balance', 'readiness.contributors.body_temperature', 'readiness.contributors.hrv_balance', 'readiness.contributors.previous_day_activity', 'readiness.contributors.previous_night', 'readiness.contributors.recovery_index', 'readiness.contributors.resting_heart_rate', 'readiness.contributors.sleep_balance', 'readiness.score', 'readiness.temperature_deviation', 'readiness.temperature_trend_deviation', 'rem_sleep_duration', 'restless', 'restle

## Prepare regressor list for lazypredict

In [6]:
removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
]

In [7]:
REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (est[0] not in removed_regressors))
]

## Run LazyRegressor on datasets

In [8]:
def lazy_regressor(X, y, test_size, random_state, regressor):
    # Create the test and train sets for the given prediction
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = random_state)
    
    models, predictions = regressor.fit(X_train, X_test, y_train, y_test)
    return models

In [9]:
# Apply the LazyRegressor on the data
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, regressors = REGRESSORS)

### LazyRegressor for the factors influencing the sleep scores given all the variables

In [10]:
# Separate the indenpendent variables from the dependent variables
y_oura = df['score']
y_sub = df['subjective_sleep_score']
X = df.drop(labels=['score', 'subjective_sleep_score'], axis=1)

#### Oura sleep scores

In [11]:
models_oura = lazy_regressor(X, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 22.23it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 90%|█████████ | 36/40 [00:01<00:00, 26.96it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 77.


100%|██████████| 40/40 [00:01<00:00, 25.90it/s]

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
Lars                                     85507.19 -274518.87 345.37   
GaussianProcessRegressor                    48.93    -152.87   8.18   
MLPRegressor                                 3.06      -5.62   1.70   
QuantileRegressor                            1.36      -0.14   0.70   
DummyRegressor                               1.32      -0.02   0.67   
LassoLars                                    1.32      -0.02   0.67   
Lasso                                        1.32      -0.02   0.67   
ElasticNet                                   1.18       0.43   0.50   
KNeighborsRegressor                          1.11       0.65   0.39   
PassiveAggressiveRegressor                   1.09       0.70   0.36   
SVR                                          1.05       0.85   0.26   
SGDRegressor                                 1.05       0.85   0.26   
NuSVR 

#### Subjective sleep scores

In [12]:
models_sub = lazy_regressor(X, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:02<00:01,  9.46it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 88%|████████▊ | 35/40 [00:02<00:00, 15.09it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 77.


100%|██████████| 40/40 [00:03<00:00, 13.09it/s]

                               Adjusted R-Squared   R-Squared    RMSE  \
Model                                                                   
Lars                                    883401.51 -2836179.60 1290.84   
LarsCV                                  882807.87 -2834273.70 1290.40   
GaussianProcessRegressor                    29.18      -89.47    7.29   
MLPRegressor                                 4.25       -9.42    2.47   
HuberRegressor                               2.89       -5.06    1.89   
LinearSVR                                    2.66       -4.34    1.77   
LinearRegression                             2.49       -3.80    1.68   
TransformedTargetRegressor                   2.49       -3.80    1.68   
PassiveAggressiveRegressor                   2.40       -3.48    1.62   
Ridge                                        2.09       -2.50    1.43   
KernelRidge                                  2.09       -2.49    1.43   
ExtraTreeRegressor                           2.01  

### LazyRegressor for the factors influencing the sleep scores given the questionnaire variables only

In [13]:
# Separate the indenpendent variables from the dependent variables
X_quest = df_quest.drop(labels=['subjective_sleep_score'], axis=1)

#### Oura sleep score

In [14]:
models_oura = lazy_regressor(X_quest, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:00<00:00, 41.58it/s]

                                        Adjusted R-Squared  \
Model                                                        
RANSACRegressor               5550509686441437569220608.00   
TransformedTargetRegressor    3945868123691236162797568.00   
LinearRegression              3945868123691236162797568.00   
KernelRidge                                         217.72   
GaussianProcessRegressor                            133.94   
MLPRegressor                                          6.54   
ExtraTreeRegressor                                    4.21   
ExtraTreesRegressor                                   3.52   
LinearSVR                                             3.17   
DecisionTreeRegressor                                 3.04   
PassiveAggressiveRegressor                            2.68   
QuantileRegressor                                     2.55   
OrthogonalMatchingPursuitCV                           2.41   
OrthogonalMatchingPursuit                             2.41   
Lasso   

#### Subjective sleep scores

In [15]:
models_sub = lazy_regressor(X_quest, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:00<00:00, 43.80it/s]

                                       Adjusted R-Squared  \
Model                                                       
RANSACRegressor               296299723676300022382592.00   
KernelRidge                                        130.76   
GaussianProcessRegressor                            93.10   
PassiveAggressiveRegressor                           6.46   
MLPRegressor                                         5.62   
DecisionTreeRegressor                                4.12   
LinearSVR                                            3.76   
ExtraTreeRegressor                                   3.54   
KNeighborsRegressor                                  3.49   
ExtraTreesRegressor                                  3.42   
SVR                                                  3.26   
HuberRegressor                                       3.26   
LinearRegression                                     3.17   
TransformedTargetRegressor                           3.17   
Ridge                   

### LazyRegressor for the factors influencing the sleep scores given the Oura variables only

In [16]:
# Separate the indenpendent variables from the dependent variables
X_oura = df_oura.drop(labels=['score'], axis=1)

#### Oura sleep score

In [17]:
models_oura = lazy_regressor(X_oura, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 32.00it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
Lars                                       122.95    -191.55  9.15        0.01
GaussianProcessRegressor                    98.34    -152.69  8.17        0.01
MLPRegressor                                 8.92     -11.50  2.33        0.09
QuantileRegressor                            1.72      -0.14  0.70        0.04
LassoLars                                    1.65      -0.02  0.67        0.01
DummyRegressor                               1.65      -0.02  0.67        0.02
Lasso                                        1.65      -0.02  0.67        0.01
ElasticNet                                   1.36       0.43  0.50        0.01
PassiveAggressiveRegressor                   1.30       0.53  0.45        0.01
ExtraTreeRegressor                           1.23       0.64  0.39        0.01
KNeighborsRegressor                          1.21   

#### Subjective sleep scores

In [18]:
models_sub = lazy_regressor(X_oura, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 21.63it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
Lars                                      1992.41   -3143.33 42.98        0.01
GaussianProcessRegressor                    58.08     -89.12  7.28        0.01
MLPRegressor                                 6.38      -7.50  2.23        0.15
RANSACRegressor                              4.66      -4.77  1.84        0.06
TransformedTargetRegressor                   4.16      -3.99  1.71        0.01
LinearRegression                             4.16      -3.99  1.71        0.01
HuberRegressor                               3.60      -3.10  1.55        0.03
LinearSVR                                    3.42      -2.83  1.50        0.01
Ridge                                        2.85      -1.93  1.31        0.01
KernelRidge                                  2.83      -1.89  1.30        0.01
PassiveAggressiveRegressor                   2.39   

### LazyRegressor for the factors influencing the average HRV given all the variables

In [19]:
# Separate the indenpendent variables from the dependent variables
y_hrv = df['average_hrv']
X_hrv = df.drop(labels=['average_hrv'], axis=1)
X_hrv_quest = df_quest
X_hrv_oura = df_oura.drop(labels=['average_hrv'], axis=1)

In [20]:
# Drop the rows for which the HRV value is not saved
for i in range(len(df['average_hrv'])):
    if pd.isna(df.iloc[i]['average_hrv']):
        y_hrv = y_hrv.drop(i)
        X_hrv = X_hrv.drop(i)
        X_hrv_quest = X_hrv_quest.drop(i)
        X_hrv_oura = X_hrv_oura.drop(i)

In [21]:
models_hrv = lazy_regressor(X_hrv, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 15.27it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 80%|████████  | 32/40 [00:01<00:00, 21.60it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 76.


100%|██████████| 40/40 [00:02<00:00, 19.84it/s]

                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
Lars                                        12.16     -35.41 114.65   
GaussianProcessRegressor                     5.25     -12.86  70.75   
MLPRegressor                                 1.95      -2.10  33.43   
DummyRegressor                               1.31      -0.01  19.10   
QuantileRegressor                            1.31      -0.01  19.09   
HuberRegressor                               1.30       0.04  18.65   
TransformedTargetRegressor                   1.28       0.08  18.23   
LinearRegression                             1.28       0.08  18.23   
NuSVR                                        1.27       0.12  17.87   
SVR                                          1.26       0.14  17.62   
PassiveAggressiveRegressor                   1.20       0.34  15.41   
KernelRidge                                  1.18       0.40  14.67   
Ridge 

### LazyRegressor for the factors influencing the average HRV given the questionnaire variables only

In [22]:
models_hrv = lazy_regressor(X_hrv_quest, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:00<00:00, 42.03it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
KernelRidge                                 19.30     -13.45 72.22        0.01
GaussianProcessRegressor                    17.07     -11.69 67.67        0.01
MLPRegressor                                10.96      -6.86 53.27        0.10
ExtraTreeRegressor                           3.02      -0.60 24.01        0.01
LinearSVR                                    2.90      -0.50 23.29        0.01
HuberRegressor                               2.61      -0.27 21.44        0.01
TransformedTargetRegressor                   2.43      -0.13 20.17        0.01
LinearRegression                             2.43      -0.13 20.17        0.01
Ridge                                        2.29      -0.02 19.17        0.01
LassoLars                                    2.28      -0.01 19.10        0.01
DummyRegressor                               2.28   

### LazyRegressor for the factors influencing the average HRV given the Oura variables only

In [23]:
models_hrv = lazy_regressor(X_hrv_oura, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 29.25it/s]


                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
Lars                                       235.26    -368.89 365.43   
GaussianProcessRegressor                     9.76     -12.84  70.68   
MLPRegressor                                 3.02      -2.19  33.93   
DummyRegressor                               1.64      -0.01  19.10   
QuantileRegressor                            1.64      -0.01  19.09   
NuSVR                                        1.52       0.18  17.23   
SVR                                          1.50       0.22  16.81   
DecisionTreeRegressor                        1.33       0.48  13.70   
LassoLars                                    1.32       0.49  13.55   
GradientBoostingRegressor                    1.31       0.51  13.26   
HuberRegressor                               1.29       0.54  12.92   
LinearRegression                             1.29       0.55  12.80   
Transf

### LazyRegressor for the factors influencing the total sleep duration given all variables

In [24]:
# Separate the indenpendent variables from the dependent variables
# For the total sleep duration, only consider the days for which a person woke up naturally
y_total_sleep = df[df['Naturally'] == 1]['total_sleep_duration']
X_total_sleep = df[df['Naturally'] == 1]
X_total_sleep = X_total_sleep.drop(labels=['total_sleep_duration', 'deep_sleep_duration', 'light_sleep_duration', 'contributors.total_sleep', 'contributors.deep_sleep', 'contributors.rem_sleep', 'rem_sleep_duration', 'time_in_bed', 'bedtime_start_delta', 'bedtime_end_delta', 'bedtime_end_seconds', 'bed_time'], axis=1)

In [25]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 70%|███████   | 28/40 [00:00<00:00, 49.40it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.
RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 8.


100%|██████████| 40/40 [00:00<00:00, 51.29it/s]

                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
GaussianProcessRegressor                     7.61    -454.88 19536.44   
LinearSVR                                    7.59    -453.76 19512.47   
MLPRegressor                                 7.56    -451.41 19462.07   
DummyRegressor                               2.02     -69.08  7659.85   
HistGradientBoostingRegressor                2.02     -69.08  7659.85   
NuSVR                                        1.95     -64.52  7406.57   
ExtraTreesRegressor                          1.90     -61.26  7219.52   
RandomForestRegressor                        1.79     -53.37  6747.01   
AdaBoostRegressor                            1.77     -52.12  6668.54   
QuantileRegressor                            1.75     -50.79  6584.75   
SVR                                          1.74     -49.92  6529.22   
Lasso                                        1.70  

### LazyRegressor for the factors influencing the total sleep duration given the questionnaire variables only

In [26]:
X_total_sleep = df_quest[df_quest['Naturally'] == 1]

In [27]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:00<00:00, 55.95it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.
RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 8.


100%|██████████| 40/40 [00:00<00:00, 64.27it/s]

                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
KernelRidge                                 21.05    -660.51 23533.72   
GaussianProcessRegressor                    14.81    -454.87 19536.26   
LinearSVR                                   14.80    -454.51 19528.45   
MLPRegressor                                14.78    -453.64 19509.97   
PassiveAggressiveRegressor                   5.83    -158.37 11551.30   
Lars                                         5.26    -139.42 10842.81   
HistGradientBoostingRegressor                3.12     -69.08  7659.85   
DummyRegressor                               3.12     -69.08  7659.85   
NuSVR                                        2.99     -64.52  7406.30   
QuantileRegressor                            2.59     -51.43  6625.44   
SVR                                          2.54     -49.92  6529.00   
KNeighborsRegressor                          2.42  

### LazyRegressor for the factors influencing the total sleep duration given the Oura variables only

In [28]:
X_total_sleep = df[df['Naturally'] == 1]
X_total_sleep = X_total_sleep.iloc[:, 0:49]
X_total_sleep = X_total_sleep.drop(labels=['total_sleep_duration', 'deep_sleep_duration', 'light_sleep_duration', 'contributors.total_sleep', 'contributors.deep_sleep', 'contributors.rem_sleep', 'rem_sleep_duration', 'time_in_bed', 'bedtime_start_delta', 'bedtime_end_delta', 'bedtime_end_seconds', 'bed_time'], axis=1)

In [29]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:00<00:00, 50.51it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.
RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 8.


100%|██████████| 40/40 [00:00<00:00, 53.54it/s]

                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
GaussianProcessRegressor                    13.66    -454.88 19536.44   
LinearSVR                                   13.63    -453.76 19512.47   
MLPRegressor                                13.59    -452.07 19476.09   
DecisionTreeRegressor                        5.47    -160.08 11612.77   
BaggingRegressor                             3.20     -78.28  8147.11   
HistGradientBoostingRegressor                2.95     -69.08  7659.85   
DummyRegressor                               2.95     -69.08  7659.85   
GradientBoostingRegressor                    2.87     -66.28  7505.04   
NuSVR                                        2.82     -64.52  7406.67   
ExtraTreesRegressor                          2.74     -61.50  7233.99   
RandomForestRegressor                        2.72     -61.01  7205.20   
QuantileRegressor                            2.43  

### LazyRegressor for the factors influencing the total awake time during the night given all variables

In [30]:
# Separate the indenpendent variables from the dependent variables
# For the total awake time, Oura also considers some time before falling asleep and some time after waking up. 
# Subtract what we can from that, which is the time before falling asleep
y_awake_time = df['awake_time'] - df['latency']
X_awake_time = df.drop(labels=['awake_time', 'time_in_bed', 'latency', 'total_sleep_duration'], axis=1)

In [31]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 21.14it/s]

                               Adjusted R-Squared      R-Squared        RMSE  \
Model                                                                          
Lars                                2422551772.38 -7522660762.77 71365345.07   
GaussianProcessRegressor                     3.73          -7.48     2396.70   
MLPRegressor                                 3.57          -7.00     2326.58   
LinearSVR                                    3.26          -6.03     2181.61   
QuantileRegressor                            1.39          -0.23      911.21   
SVR                                          1.39          -0.22      910.38   
NuSVR                                        1.37          -0.15      881.46   
DummyRegressor                               1.36          -0.10      863.92   
KNeighborsRegressor                          1.30           0.06      796.48   
PoissonRegressor                             1.27           0.15      759.90   
GammaRegressor                          

### LazyRegressor for the factors influencing the total awake time during the night given the questionnaire variables only

In [32]:
X_awake_time = df_quest

In [33]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:00<00:00, 42.46it/s]

                               Adjusted R-Squared  R-Squared    RMSE  \
Model                                                                  
KernelRidge                                 13.14      -8.59 2547.45   
MLPRegressor                                11.54      -7.32 2373.72   
LinearSVR                                   11.11      -6.98 2324.97   
GaussianProcessRegressor                     8.80      -5.15 2041.32   
Lars                                         3.18      -0.72 1080.35   
ExtraTreeRegressor                           3.14      -0.69 1070.23   
TransformedTargetRegressor                   2.85      -0.46  995.56   
LinearRegression                             2.85      -0.46  995.56   
DecisionTreeRegressor                        2.85      -0.46  995.37   
HuberRegressor                               2.81      -0.43  984.33   
Lasso                                        2.80      -0.42  980.19   
RANSACRegressor                              2.77      -0.40  97

### LazyRegressor for the factors influencing the total awake time during the night given the Oura variables only

In [34]:
X_awake_time = df_oura.drop(labels=['awake_time', 'time_in_bed', 'latency', 'total_sleep_duration'], axis=1)

In [35]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 40/40 [00:01<00:00, 27.79it/s]

                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
Lars                                      1594.39   -2263.29 39153.26   
GaussianProcessRegressor                     6.95      -7.46  2392.60   
MLPRegressor                                 6.66      -7.04  2332.84   
LinearSVR                                    5.95      -6.03  2181.61   
QuantileRegressor                            1.86      -0.23   911.21   
SVR                                          1.86      -0.22   910.31   
KNeighborsRegressor                          1.85      -0.21   906.51   
NuSVR                                        1.81      -0.15   881.08   
DummyRegressor                               1.78      -0.10   863.92   
GammaRegressor                               1.55       0.22   726.49   
TweedieRegressor                             1.45       0.36   660.47   
ElasticNetCV                                 1.43  